In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from google.colab import files
uploaded = files.upload()

Saving Dataset_Enfermedades.csv to Dataset_Enfermedades.csv


In [ ]:
# 1. Cargar el dataset de enfermedades
df = pd.read_csv('Dataset_Enfermedades.csv')
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# 2. Preparar variables para el modelo
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

#  **NUEVO: Filtrar enfermedades según los síntomas iniciales del usuario**
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

# Verificar si los síntomas ingresados existen en el dataset
sintomas_validos = [s for s in sintomas_usuario if s in symptom_cols]

df_filtrado = df.copy()  # Copia del dataset original

if sintomas_validos:
    # Mantener enfermedades que coincidan con AL MENOS el 50% de los síntomas ingresados
    df_filtrado['suma_sintomas'] = df_filtrado[sintomas_validos].sum(axis=1)
    df_filtrado = df_filtrado[df_filtrado['suma_sintomas'] >= (len(sintomas_validos) / 2)]


    # Si el filtro es muy estricto y quedan pocas enfermedades, ampliar el criterio
    if len(df_filtrado) < 3:
        print(" Pocas coincidencias. Se ampliará la búsqueda.")
        df_filtrado = df[df[sintomas_validos].sum(axis=1) > 0]

    df_filtrado = df_filtrado.drop(columns=['suma_sintomas'])

    print(f" Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")
else:
    print(" No se encontraron síntomas válidos. Se usará el dataset completo.")

# 3. Codificar la variable objetivo
le = LabelEncoder()
df['enfermedad_cod'] = le.fit_transform(df['nombre de la enfermedad'])

# 4. Crear variables de entrada (X) y salida (y)
X = df_filtrado[symptom_cols + risk_cols]  # Solo enfermedades filtradas
y = le.fit_transform(df_filtrado['nombre de la enfermedad'])  # Solo enfermedades filtradas


#  **NUEVO: Normalizar datos para evitar sesgo en factores de riesgo**
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. Entrenar el modelo de Random Forest con datos normalizados
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=42)
clf.fit(X_scaled, y)


# 6. Solicitar factores de riesgo al usuario
edad = int(input("Ingrese su edad: "))
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = float(input("Ingrese su peso en kilogramos: "))
altura = float(input("Ingrese su altura en metros: "))

# 7. Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# 8. Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})

# 9. Asignar género e IMC
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

#  **NUEVO: Seleccionar síntomas más comunes dentro del dataset filtrado**
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()

# Limitar preguntas a los síntomas más frecuentes en las enfermedades filtradas
num_preguntas = min(40, max(20, len(sintomas_comunes)))  # Entre 20 y 40 preguntas
top_symptoms = sintomas_comunes[:num_preguntas]

# 11. Preguntar al usuario sobre los síntomas más relevantes
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
questions_asked = 0
for symptom in top_symptoms:
    if questions_asked >= 20:  # Aumentamos la cantidad de preguntas para mayor precisión
        break
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        resp = input(f"Por favor responda con 's' o 'n'. ¿Presenta '{symptom}'? ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0
    questions_asked += 1

# 12. Generar el vector de entrada para el modelo
user_vector = [user_answers[feat] for feat in X.columns]

# 13. Obtener probabilidades de enfermedades con el modelo
user_vector_scaled = scaler.transform([user_vector])  # Normalizar antes de predecir
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# 14. Mostrar los resultados finales
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")

Ingrese los síntomas que tiene (separados por comas): erupción cutánea, dolor de cabeza, dolor detrás de los ojos, dolor muscular
🔎 Se han identificado 3 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 20
Ingrese su género (M/F): m
Ingrese su peso en kilogramos: 70
Ingrese su altura en metros: 1.73

Su IMC es 23.39, lo que corresponde a 'peso normal'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'escalofríos'? (s/n): n
¿Presenta 'dolor de cabeza'? (s/n): s
¿Presenta 'dolor muscular'? (s/n): s
¿Presenta 'fatiga'? (s/n): s
¿Presenta 'fiebre alta'? (s/n): s
¿Presenta 'tos'? (s/n): n
¿Presenta 'dolor de garganta'? (s/n): n
¿Presenta 'dolor de cabeza intenso'? (s/n): s
¿Presenta 'pérdida del olfato y gusto'? (s/n): s
¿Presenta 'dolor muscular y en articulaciones'? (s/n): s
¿Presenta 'dificultad respiratoria'? (s/n): s
¿Presenta 'erupción cutánea'? (s/n): s
¿Presenta 'náuseas'? (s/n): s
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'congestión nasal

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el dataset
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

# Solicitar síntomas al usuario
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

# Mejor coincidencia de síntomas
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols):
    sintomas_validos = []
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=0.8)
        if match:
            sintomas_validos.append(match[0])  # Usa el síntoma más cercano encontrado
    return sintomas_validos

sintomas_validos = encontrar_sintomas_validos(sintomas_usuario, symptom_cols)

# Filtrar enfermedades
if sintomas_validos:
    df['match_score'] = df[sintomas_validos].sum(axis=1) / df[symptom_cols].sum(axis=1)
    df_filtrado = df[df['match_score'] >= 0.5].drop(columns=['match_score'])
    if len(df_filtrado) < 3:
        df_filtrado = df[df[sintomas_validos].sum(axis=1) > 0]
        df_filtrado = df_filtrado.copy()
else:
    df_filtrado = df.copy()

print(f" Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre de la enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Entrenar modelo Random Forest con max_depth
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, random_state=42)
clf.fit(X_scaled, y)

# Solicitar datos del usuario con validación
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kilogramos: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes en el dataset filtrado
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
top_symptoms = sintomas_comunes[:min(40, max(20, len(sintomas_comunes)))]

# Preguntar sobre síntomas con mejor validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms[:20]:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        print("⚠️ Entrada inválida. Por favor, ingrese 's' para sí o 'n' para no.")
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

# Crear vector de entrada con nombres de columnas
user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
user_vector_scaled = scaler.transform(user_vector_df)

# Obtener probabilidades de enfermedades
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar resultados
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")



Ingrese los síntomas que tiene (separados por comas): dolor de cabeza y tos
 Se han identificado 4 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kilogramos: 125
Ingrese su altura en metros: 2.50

Su IMC es 20.00, lo que corresponde a 'peso normal'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n): S
¿Presenta 'dolor de cabeza'? (s/n): N
¿Presenta 'escalofríos'? (s/n): N
¿Presenta 'dolor de garganta'? (s/n): S
¿Presenta 'dificultad respiratoria'? (s/n): S
¿Presenta 'congestión nasal'? (s/n): S
¿Presenta 'tos seca'? (s/n): S
¿Presenta 'dolor muscular'? (s/n): N
¿Presenta 'pérdida del olfato y gusto'? (s/n): S
¿Presenta 'secreción nasal acuosa'? (s/n): S
¿Presenta 'estornudos'? (s/n): S
¿Presenta 'fiebre'? (s/n): S
¿Presenta 'fiebre alta'? (s/n): S
¿Presenta 'tos'? (s/n): S
¿Presenta 'mareos'? (s/n): S
¿Presenta 'visión borrosa'? (s/n): S
¿Presenta 'dolor en el pecho'? (s/n): S

In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el dataset
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

# Función para encontrar coincidencias de síntomas
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=0.8)
        if match:
            sintomas_validos.append(match[0])  # Usa el síntoma más cercano encontrado
            coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols)

# Mostrar coincidencias encontradas
print("\nCoincidencias encontradas:")
for original, match in coincidencias.items():
    print(f"'{original}' coincide con '{match}' en el dataset.")

# Filtrar enfermedades SOLO con los síntomas ingresados
if sintomas_validos:
    df_filtrado = df[df[sintomas_validos].sum(axis=1) > 0].copy()
else:
    df_filtrado = df.copy()

print(f" Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre de la enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Entrenar modelo Random Forest con max_depth
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, random_state=42)
clf.fit(X_scaled, y)

# Solicitar datos del usuario con validación
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kilogramos: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes en el dataset filtrado
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
top_symptoms = sintomas_comunes[:min(40, max(20, len(sintomas_comunes)))]

# Preguntar sobre síntomas con mejor validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms[:20]:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        print("⚠️ Entrada inválida. Por favor, ingrese 's' para sí o 'n' para no.")
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

# Crear vector de entrada con nombres de columnas
user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
user_vector_scaled = scaler.transform(user_vector_df)

# Obtener probabilidades de enfermedades
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar resultados
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese los síntomas que tiene (separados por comas): tos y fiebre

Coincidencias encontradas:
 Se han identificado 100 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kilogramos: 100
Ingrese su altura en metros: 1.82

Su IMC es 30.19, lo que corresponde a 'obesidad'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n): s
¿Presenta 'pérdida de peso'? (s/n): n
¿Presenta 'náuseas'? (s/n): n
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'dificultad respiratoria'? (s/n): n
¿Presenta 'dolor abdominal'? (s/n): s
¿Presenta 'visión borrosa'? (s/n): n
¿Presenta 'ictericia'? (s/n): n
¿Presenta 'picazón'? (s/n): s
¿Presenta 'hinchazón'? (s/n): n
¿Presenta 'dolor en el pecho'? (s/n): n
¿Presenta 'dificultad para respirar'? (s/n): s
¿Presenta 'dolor de cabeza'? (s/n): s
¿Presenta 'fiebre alta'? (s/n): s
¿Presenta 'sudoración'? (s/n): n
¿Presenta 'enrojecimiento'? (s/n): s
¿Presenta 'erupción cutánea

In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el dataset
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

# Función para encontrar coincidencias de síntomas
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=0.8)
        if match:
            sintomas_validos.append(match[0])  # Usa el síntoma más cercano encontrado
            coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols)

# Mostrar coincidencias encontradas
print("\nCoincidencias encontradas:")
for original, match in coincidencias.items():
    print(f"'{original}' coincide con '{match}' en el dataset.")

# Filtrar enfermedades que contengan TODOS los síntomas ingresados
if sintomas_validos:
    df_filtrado = df.copy()
    for sintoma in sintomas_validos:
        df_filtrado = df_filtrado[df_filtrado[sintoma] == 1]
else:
    df_filtrado = df.copy()

print(f"\nSíntomas utilizados para filtrar enfermedades: {sintomas_validos}")
print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre de la enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Entrenar modelo Random Forest con max_depth
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, random_state=42)
clf.fit(X_scaled, y)

# Solicitar datos del usuario con validación
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kilogramos: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes en el dataset filtrado
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
top_symptoms = sintomas_comunes[:min(40, max(20, len(sintomas_comunes)))]

# Preguntar sobre síntomas con mejor validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms[:20]:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        print("⚠️ Entrada inválida. Por favor, ingrese 's' para sí o 'n' para no.")
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

# Crear vector de entrada con nombres de columnas
user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
user_vector_scaled = scaler.transform(user_vector_df)

# Obtener probabilidades de enfermedades
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar resultados
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese los síntomas que tiene (separados por comas): tos,dolor de cabeza,fiebre

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'dolor de cabeza' coincide con 'dolor de cabeza' en el dataset.
'fiebre' coincide con 'fiebre' en el dataset.

Síntomas utilizados para filtrar enfermedades: ['tos', 'dolor de cabeza', 'fiebre']
Se han identificado 1 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kilogramos: 1.82
Ingrese su altura en metros: 100

Su IMC es 0.00, lo que corresponde a 'desnutricion'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'dolor de cabeza'? (s/n): s
¿Presenta 'pérdida del olfato y gusto'? (s/n): n
¿Presenta 'escalofríos'? (s/n): s
¿Presenta 'fatiga'? (s/n): n
¿Presenta 'dificultad respiratoria'? (s/n): s
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'dolor muscular'? (s/n): n
¿Presenta 'tos'? (s/n): s
¿Presenta 'presión arterial alta'? (s/n): n
¿Presenta 'presi

In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el dataset
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

# Función para encontrar coincidencias de síntomas
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=0.8)
        if match:
            sintomas_validos.append(match[0])  # Usa el síntoma más cercano encontrado
            coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols)

# Mostrar coincidencias encontradas
print("\nCoincidencias encontradas:")
for original, match in coincidencias.items():
    print(f"'{original}' coincide con '{match}' en el dataset.")

# Filtrar enfermedades que contengan TODOS los síntomas ingresados
if sintomas_validos:
    df_filtrado = df.copy()
    for sintoma in sintomas_validos:
        df_filtrado = df_filtrado[df_filtrado[sintoma] == 1]
else:
    df_filtrado = df.copy()

print(f"\nSíntomas utilizados para filtrar enfermedades: {sintomas_validos}")
print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre de la enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Entrenar modelo Random Forest con max_depth
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, random_state=42)
clf.fit(X_scaled, y)

# Solicitar datos del usuario con validación
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kilogramos: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes en el dataset filtrado excluyendo los ya ingresados
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
sintomas_comunes = [s for s in sintomas_comunes if s not in sintomas_validos]
top_symptoms = sintomas_comunes[:min(40, max(20, len(sintomas_comunes)))]

# Preguntar sobre síntomas con mejor validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms[:20]:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        print("⚠️ Entrada inválida. Por favor, ingrese 's' para sí o 'n' para no.")
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

# Crear vector de entrada con nombres de columnas
user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
user_vector_scaled = scaler.transform(user_vector_df)

# Obtener probabilidades de enfermedades
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar resultados
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese los síntomas que tiene (separados por comas): tos, fiebre

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'fiebre' coincide con 'fiebre' en el dataset.

Síntomas utilizados para filtrar enfermedades: ['tos', 'fiebre']
Se han identificado 1 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kilogramos: 1.82
Ingrese su altura en metros: 0.7

Su IMC es 3.71, lo que corresponde a 'desnutricion'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'dolor de cabeza'? (s/n): s
¿Presenta 'pérdida del olfato y gusto'? (s/n): n
¿Presenta 'escalofríos'? (s/n): n
¿Presenta 'fatiga'? (s/n): s
¿Presenta 'dificultad respiratoria'? (s/n): n
¿Presenta 'dolor muscular'? (s/n): n
¿Presenta 'presión arterial alta'? (s/n): s
¿Presenta 'presión arterial baja'? (s/n): n
¿Presenta 'orina con sangre'? (s/n): n
¿Presenta 'orina oscura'? (s/n): n
¿Presenta 'orina turbia o con mal olor'? (s/n): n
¿

In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el dataset
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

# Función para encontrar coincidencias de síntomas
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=0.8)
        if match:
            sintomas_validos.append(match[0])  # Usa el síntoma más cercano encontrado
            coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols)

# Mostrar coincidencias encontradas
print("\nCoincidencias encontradas:")
for original, match in coincidencias.items():
    print(f"'{original}' coincide con '{match}' en el dataset.")

# Filtrar enfermedades que contengan TODOS los síntomas ingresados
if sintomas_validos:
    df_filtrado = df.copy()
    for sintoma in sintomas_validos:
        df_filtrado = df_filtrado[df_filtrado[sintoma] == 1]
else:
    df_filtrado = df.copy()

print(f"\nSíntomas utilizados para filtrar enfermedades: {sintomas_validos}")
print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre de la enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Entrenar modelo Random Forest con max_depth
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, random_state=42)
clf.fit(X_scaled, y)

# Solicitar datos del usuario con validación
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kilogramos: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes en el dataset filtrado excluyendo los ya ingresados
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
sintomas_comunes = [s for s in sintomas_comunes if s not in sintomas_validos]
top_symptoms = sintomas_comunes[:min(40, max(20, len(sintomas_comunes)))]

# Preguntar sobre síntomas con mejor validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms[:20]:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        print("⚠️ Entrada inválida. Por favor, ingrese 's' para sí o 'n' para no.")
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

    # Verificar si hay una probabilidad mayor al 80% después de cada respuesta
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]
    top3_idx = np.argsort(probs)[-3:][::-1]

    if max(probs) >= 0.80:
        break

# Mostrar resultados
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese los síntomas que tiene (separados por comas): tos,fiebre

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'fiebre' coincide con 'fiebre' en el dataset.

Síntomas utilizados para filtrar enfermedades: ['tos', 'fiebre']
Se han identificado 1 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kilogramos: 100
Ingrese su altura en metros: 1.82

Su IMC es 30.19, lo que corresponde a 'obesidad'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'dolor de cabeza'? (s/n): s

*** Diagnóstico Final ***

1. **COVID-19** – Probabilidad: 100.0%
   *Descripción:* Enfermedad viral causada por el coronavirus SARS-CoV-2, que afecta principalmente los pulmones. Puede ser leve o grave.
   *Tratamiento:* Cuidados de soporte, oxigenoterapia en casos graves, antivirales, corticosteroides según la severidad.



In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el dataset
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

# Función para encontrar coincidencias de síntomas
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=0.8)
        if match:
            sintomas_validos.append(match[0])  # Usa el síntoma más cercano encontrado
            coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols)

# Mostrar coincidencias encontradas
print("\nCoincidencias encontradas:")
for original, match in coincidencias.items():
    print(f"'{original}' coincide con '{match}' en el dataset.")

# Filtrar enfermedades con al menos 80% de coincidencia con los síntomas ingresados
if sintomas_validos:
    df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
    df_filtrado = df[df['match_score'] >= 0.8].drop(columns=['match_score'])
else:
    df_filtrado = df.copy()

print(f"\nSíntomas utilizados para filtrar enfermedades: {sintomas_validos}")
print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre de la enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ajustar max_depth dinámicamente
max_depth_value = max(1, min(10, len(df_filtrado) // 2))
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=max_depth_value, random_state=42)
clf.fit(X_scaled, y)

# Solicitar datos del usuario con validación
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kilogramos: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes en el dataset filtrado excluyendo los ya ingresados
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
sintomas_comunes = [s for s in sintomas_comunes if s not in sintomas_validos]
top_symptoms = sintomas_comunes[:min(40, max(20, len(sintomas_comunes)))]

# Preguntar sobre síntomas con mejor validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms[:20]:
    resp = input(f"¿Presenta '{symptom}'? (s/n o 'salir' para terminar): ").strip().lower()
    if resp == "salir":
        print("\n⏹️ Diagnóstico interrumpido por el usuario. Mostrando predicción actual...\n")
        break
    while resp not in ["s", "n", "si", "no"]:
        print("⚠️ Entrada inválida. Por favor, ingrese 's' para sí, 'n' para no, o 'salir' para terminar.")
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

    # Verificar si hay una probabilidad mayor al 80% después de cada respuesta
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]
    top3_idx = np.argsort(probs)[-3:][::-1]

    # print("\nEstado actual de predicción:")
    for rank, idx in enumerate(top3_idx, start=1):
        disease_code = clf.classes_[idx]
        disease = le.inverse_transform([disease_code])[0]
        prob_percent = probs[idx] * 100
        # print(f"{rank}. {disease} – {prob_percent:.1f}%")

    if max(probs) >= 0.80:
        break

# Mostrar resultados
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")



Ingrese los síntomas que tiene (separados por comas): tos

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.

Síntomas utilizados para filtrar enfermedades: ['tos']
Se han identificado 3 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kilogramos: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n o 'salir' para terminar): s
¿Presenta 'dificultad respiratoria'? (s/n o 'salir' para terminar): s
¿Presenta 'opresión en el pecho'? (s/n o 'salir' para terminar): n
¿Presenta 'erupción cutánea'? (s/n o 'salir' para terminar): s
¿Presenta 'escalofríos'? (s/n o 'salir' para terminar): n
¿Presenta 'conjuntivitis'? (s/n o 'salir' para terminar): s
¿Presenta 'sibilancias'? (s/n o 'salir' para terminar): b
⚠️ Entrada inválida. Por favor, ingrese 's' para sí, 'n' para no, o 'salir' para termi

In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar el dataset
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de columnas

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre de la enfermedad', 'breve descripción', 'tratamiento'] + risk_cols]

# Función para encontrar coincidencias de síntomas
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=0.8)
        if match:
            sintomas_validos.append(match[0])  # Usa el síntoma más cercano encontrado
            coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
sintomas_usuario = input("Ingrese los síntomas que tiene (separados por comas): ").lower().strip()
sintomas_usuario = [s.strip() for s in sintomas_usuario.split(",")]

sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols)

# Mostrar coincidencias encontradas
print("\nCoincidencias encontradas:")
for original, match in coincidencias.items():
    print(f"'{original}' coincide con '{match}' en el dataset.")

# Filtrar enfermedades con al menos 80% de coincidencia con los síntomas ingresados
if sintomas_validos:
    df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
    df_filtrado = df[df['match_score'] >= 0.8].drop(columns=['match_score'])
else:
    df_filtrado = df.copy()

print(f"\nSíntomas utilizados para filtrar enfermedades: {sintomas_validos}")
print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre de la enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ajustar max_depth dinámicamente
max_depth_value = max(1, min(10, len(df_filtrado) // 2))
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=max_depth_value, random_state=42)
clf.fit(X_scaled, y)

# Solicitar datos del usuario con validación
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kilogramos: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes en el dataset filtrado excluyendo los ya ingresados
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
sintomas_comunes = [s for s in sintomas_comunes if s not in sintomas_validos]
top_symptoms = sintomas_comunes[:min(40, max(20, len(sintomas_comunes)))]

# Preguntar sobre síntomas con mejor validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms[:20]:
    resp = input(f"¿Presenta '{symptom}'? (s/n o 'salir' para terminar): ").strip().lower()
    if resp == "salir":
        print("\n⏹️ Diagnóstico interrumpido por el usuario. Mostrando predicción actual...\n")
        break
    while resp not in ["s", "n", "si", "no"]:
        print("⚠️ Entrada inválida. Por favor, ingrese 's' para sí, 'n' para no, o 'salir' para terminar.")
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

    # Verificar si hay una probabilidad mayor al 80% después de cada respuesta
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]
    top3_idx = np.argsort(probs)[-3:][::-1]

    if max(probs) >= 0.80:
        break

# Mostrar resultados
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre de la enfermedad'] == disease, 'breve descripción'].values[0]
    treat = df.loc[df['nombre de la enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")



Ingrese los síntomas que tiene (separados por comas): tos

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.

Síntomas utilizados para filtrar enfermedades: ['tos']
Se han identificado 3 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kilogramos: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n o 'salir' para terminar): s
¿Presenta 'dificultad respiratoria'? (s/n o 'salir' para terminar): s
¿Presenta 'opresión en el pecho'? (s/n o 'salir' para terminar): s
¿Presenta 'erupción cutánea'? (s/n o 'salir' para terminar): n
¿Presenta 'escalofríos'? (s/n o 'salir' para terminar): s
¿Presenta 'conjuntivitis'? (s/n o 'salir' para terminar): n
¿Presenta 'sibilancias'? (s/n o 'salir' para terminar): n
¿Presenta 'fiebre'? (s/n o 'salir' para terminar): s
¿Presenta 'dolor muscular'? (s/n 

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Manejo de valores nulos
df.fillna(0, inplace=True)

# Función para encontrar síntomas válidos con un umbral ajustable
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
        if match:
            sintomas_validos.append(match[0])
            coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
while True:
    entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
    sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)  # No dividir por espacios para mantener frases completas
    sintomas_usuario = [s.strip() for s in sintomas_usuario if s]
    sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85)

    print("\nCoincidencias encontradas:")
    for original, match in coincidencias.items():
        print(f"'{original}' coincide con '{match}' en el dataset.")

    # Filtrar enfermedades relevantes
    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
        df_filtrado = df[df['match_score'] >= 0.6].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    # Verificar si hay enfermedades relacionadas
    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas con los síntomas ingresados. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos solo si hay datos disponibles
if len(X) > 0:
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest
clf = RandomForestClassifier(n_estimators=100, max_depth=5, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas más comunes
sintomas_comunes = df_filtrado[symptom_cols].mean().sort_values(ascending=False).index.tolist()
top_symptoms = [s for s in sintomas_comunes if s not in sintomas_validos][:20]

# Preguntar sobre síntomas relevantes con validación
print("\nPor favor responda con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms:
    while True:
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
        if resp in ['s', 'si', 'n', 'no']:
            user_answers[symptom] = 1 if resp in ['s', 'si'] else 0
            break
        else:
            print("⚠️ Respuesta inválida. Por favor ingrese 's' para sí o 'n' para no.")

# Cargar modelo entrenado y realizar predicción
clf = joblib.load("modelo_random_forest.pkl")
scaler = joblib.load("scaler.pkl")
user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
user_vector_scaled = scaler.transform(user_vector_df)
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar siempre las 3 mejores predicciones
print("\n*** Diagnóstico Final ***")
num_diagnosticos = min(3, len(top3_idx))
for rank, idx in enumerate(top3_idx[:num_diagnosticos], start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease, 'breve_descripción'].values[0]
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")

Ingrese sus síntomas: tos, dolor de cabeza

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'dolor de cabeza' coincide con 'dolor_de_cabeza' en el dataset.
Se han identificado 1 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda con 's' (sí) o 'n' (no):

¿Presenta 'dolor_muscular'? (s/n): s
¿Presenta 'dificultad_respiratoria'? (s/n): n
¿Presenta 'fatiga'? (s/n): s
¿Presenta 'pérdida_del_olfato_y_gusto'? (s/n): n
¿Presenta 'escalofríos'? (s/n): n
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'orina_turbia_o_con_mal_olor'? (s/n): n
¿Presenta 'orina_oscura'? (s/n): n
¿Presenta 'palidez'? (s/n): n
¿Presenta 'palpitaciones'? (s/n): n
¿Presenta 'orina_con_sangre'? (s/n): n
¿Presenta 'parches_rojos_con_escamas_plateadas'? (s/n): n
¿Presenta 'opresión_en_el_pecho'? (s/n): n
¿Presenta 'pensamiento_desorganizado'? (

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Función optimizada para encontrar síntomas válidos
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        s = s.lower().strip()
        if s in symptom_cols:
            sintomas_validos.append(s)
            coincidencias[s] = s
        else:
            match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
            if match:
                sintomas_validos.append(match[0])
                coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
while True:
    entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
    sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)
    sintomas_usuario = [s.strip() for s in sintomas_usuario if s]
    sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85)

    print("\nCoincidencias encontradas:")
    for original, match in coincidencias.items():
        print(f"'{original}' coincide con '{match}' en el dataset.")

    # Filtrar enfermedades relevantes con umbral dinámico
    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
        threshold = max(0.5, df['match_score'].quantile(0.5))  # Ajusta el umbral dinámicamente
        df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos con MinMaxScaler
if len(X) > 0:
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest sin validación cruzada
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas adicionales más comunes en el dataset filtrado
sintomas_comunes = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
sintomas_comunes = [s for s in sintomas_comunes if s not in sintomas_validos][:20]

# Preguntar sobre síntomas adicionales
print("\nPor favor responda con 's' (sí) o 'n' (no):\n")
for symptom in sintomas_comunes[:20]:
    while True:
        resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
        if resp in ['s', 'si', 'n', 'no']:
            user_answers[symptom] = 1 if resp in ['s', 'si'] else 0
            break
        else:
            print("⚠️ Respuesta inválida. Por favor ingrese 's' para sí o 'n' para no.")

# Cargar modelo entrenado y realizar predicción
clf = joblib.load("modelo_random_forest.pkl")
scaler = joblib.load("scaler.pkl")
user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
user_vector_scaled = scaler.transform(user_vector_df)
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar siempre las 3 mejores predicciones
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease, 'breve_descripción'].values[0]
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese sus síntomas: dolor de cabeza, tos

Coincidencias encontradas:
'dolor de cabeza' coincide con 'dolor_de_cabeza' en el dataset.
'tos' coincide con 'tos' en el dataset.
Se han identificado 6 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n): s
¿Presenta 'dificultad_respiratoria'? (s/n): s
¿Presenta 'tos_seca'? (s/n): s
¿Presenta 'congestión_nasal'? (s/n): s
¿Presenta 'dolor_de_garganta'? (s/n): s
¿Presenta 'fiebre_alta'? (s/n): s
¿Presenta 'escalofríos'? (s/n): n
¿Presenta 'dolor_muscular'? (s/n): n
¿Presenta 'conjuntivitis'? (s/n): n
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'mareos'? (s/n): n
¿Presenta 'pérdida_del_olfato_y_gusto'? (s/n): s
¿Presenta 'sibilancias'? (s/n): n
¿Presenta 'visión_borrosa'? (s/n): n
¿Presenta 'secreción_nasal_acuosa'? (s/n): s
¿Prese

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Función optimizada para encontrar síntomas válidos
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        s = s.lower().strip()
        if s in symptom_cols:
            sintomas_validos.append(s)
            coincidencias[s] = s
        else:
            match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
            if match:
                sintomas_validos.append(match[0])
                coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
while True:
    entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
    sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)
    sintomas_usuario = [s.strip() for s in sintomas_usuario if s]
    sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85)

    print("\nCoincidencias encontradas:")
    for original, match in coincidencias.items():
        print(f"'{original}' coincide con '{match}' en el dataset.")

    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
        threshold = max(0.5, df['match_score'].quantile(0.7))
        df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos con MinMaxScaler
if len(X) > 0:
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest sin validación cruzada
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas relevantes para preguntas
top_symptoms = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
top_symptoms = [s for s in top_symptoms if s not in sintomas_validos][:20]

# Preguntar sobre síntomas con validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        resp = input(f"⚠️ Entrada inválida. ¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

    # Verificar si alguna enfermedad supera el 80% de probabilidad
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]
    if max(probs) >= 0.80:
        break

# Obtener las 3 enfermedades más probables
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar diagnóstico final
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease, 'breve_descripción'].values[0]
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese sus síntomas: tos, dolor de cabeza

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'dolor de cabeza' coincide con 'dolor_de_cabeza' en el dataset.
Se han identificado 6 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n): s
¿Presenta 'dificultad_respiratoria'? (s/n): s
¿Presenta 'tos_seca'? (s/n): s
¿Presenta 'congestión_nasal'? (s/n): s
¿Presenta 'dolor_de_garganta'? (s/n): s
¿Presenta 'fiebre_alta'? (s/n): s
¿Presenta 'escalofríos'? (s/n): n
¿Presenta 'dolor_muscular'? (s/n): n
¿Presenta 'conjuntivitis'? (s/n): n
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'mareos'? (s/n): n
¿Presenta 'pérdida_del_olfato_y_gusto'? (s/n): s
¿Presenta 'sibilancias'? (s/n): n
¿Presenta 'visión_borrosa'? (s/n): n
¿Presenta 'secreción_nasal

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Función optimizada para encontrar síntomas válidos
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        s = s.lower().strip()
        if s in symptom_cols:
            sintomas_validos.append(s)
            coincidencias[s] = s
        else:
            match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
            if match:
                sintomas_validos.append(match[0])
                coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
while True:
    intentos = 3
    while intentos > 0:
        entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
        sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)
        sintomas_usuario = [s.strip() for s in sintomas_usuario if s]
        sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.70)

        if sintomas_validos:
            break
        else:
            intentos -= 1
            if intentos > 0:
                print(f"⚠️ No se encontraron coincidencias. Intente nuevamente ({intentos} intentos restantes).")

    print("\nCoincidencias encontradas:")
    for original, match in coincidencias.items():
        print(f"'{original}' coincide con '{match}' en el dataset.")

    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
        threshold = max(0.5, df['match_score'].quantile(0.7))
        df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos con MinMaxScaler
if len(X) > 0:
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest sin validación cruzada
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas relevantes para preguntas
top_symptoms = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
top_symptoms = [s for s in top_symptoms if s not in sintomas_validos][:20]

# Preguntar sobre síntomas con validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        resp = input(f"⚠️ Entrada inválida. ¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

    # Verificar si alguna enfermedad supera el 80% de probabilidad
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]
    if max(probs) >= 0.70:
        break

# Obtener las 3 enfermedades más probables
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Mostrar diagnóstico final
print("\n*** Diagnóstico Final ***")
for rank, idx in enumerate(top3_idx, start=1):
    disease_code = clf.classes_[idx]
    disease = le.inverse_transform([disease_code])[0]
    prob_percent = probs[idx] * 100
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease, 'breve_descripción'].values[0]
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease, 'tratamiento'].values[0]
    print(f"\n{rank}. **{disease}** – Probabilidad: {prob_percent:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese sus síntomas: tos, dolor de cabeza

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'dolor de cabeza' coincide con 'dolor_de_cabeza' en el dataset.
Se han identificado 6 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n): s
¿Presenta 'dificultad_respiratoria'? (s/n): s
¿Presenta 'dolor_muscular'? (s/n): n
¿Presenta 'congestión_nasal'? (s/n): s
¿Presenta 'dolor_de_garganta'? (s/n): s
¿Presenta 'escalofríos'? (s/n): s
¿Presenta 'tos_seca'? (s/n): s
¿Presenta 'fiebre_alta'? (s/n): s
¿Presenta 'pérdida_del_olfato_y_gusto'? (s/n): s
¿Presenta 'sibilancias'? (s/n): n
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'mareos'? (s/n): n
¿Presenta 'dolor_en_el_pecho'? (s/n): n
¿Presenta 'estornudos'? (s/n): s
¿Presenta 'secreción_nasal

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Función optimizada para encontrar síntomas válidos
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        s = s.lower().strip()
        if s in symptom_cols:
            sintomas_validos.append(s)
            coincidencias[s] = s
        else:
            match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
            if match:
                sintomas_validos.append(match[0])
                coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
while True:
    intentos = 3
    while intentos > 0:
        entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
        sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)
        sintomas_usuario = [s.strip() for s in sintomas_usuario if s]
        sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.70)

        if sintomas_validos:
            break
        else:
            intentos -= 1
            if intentos > 0:
                print(f"⚠️ No se encontraron coincidencias. Intente nuevamente ({intentos} intentos restantes).")

    print("\nCoincidencias encontradas:")
    for original, match in coincidencias.items():
        print(f"'{original}' coincide con '{match}' en el dataset.")

    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
        threshold = max(0.5, df['match_score'].quantile(0.7))
        df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos con MinMaxScaler
if len(X) > 0:
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest sin validación cruzada
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar síntomas relevantes para preguntas
top_symptoms = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()
top_symptoms = [s for s in top_symptoms if s not in sintomas_validos][:20]

# Preguntar sobre síntomas con validación
print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")
for symptom in top_symptoms:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()
    while resp not in ["s", "n", "si", "no"]:
        resp = input(f"⚠️ Entrada inválida. ¿Presenta '{symptom}'? (s/n): ").strip().lower()
    user_answers[symptom] = 1 if resp in ["s", "si"] else 0

    # Verificar si alguna enfermedad supera el 80% de probabilidad
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]
    if max(probs) >= 0.70:
        break

# Obtener las 3 enfermedades más probables
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Lista de enfermedades finales y sus síntomas asociados
final_diseases = []
for idx in top3_idx:
    disease_code = clf.classes_[idx]
    disease_name = le.inverse_transform([disease_code])[0]

    # Obtener los síntomas reales de la enfermedad en el dataset
    disease_symptoms = df.loc[df['nombre_de_la_enfermedad'] == disease_name, symptom_cols].iloc[0]
    symptoms_present = disease_symptoms[disease_symptoms == 1].index.tolist()

    # Contar coincidencias con los síntomas ingresados por el usuario
    matched_symptoms = sum(user_answers[symptom] for symptom in symptoms_present)
    total_symptoms = len(symptoms_present)
    probability_match = (matched_symptoms / total_symptoms) * 100 if total_symptoms > 0 else 0

    final_diseases.append({
        "nombre": disease_name,
        "probabilidad_modelo": probs[idx] * 100,
        "probabilidad_coincidencias": probability_match,
        "síntomas_coincididos": matched_symptoms,
        "total_síntomas": total_symptoms
    })

# Mostrar diagnóstico final con ambos cálculos de probabilidad
print("\n*** Diagnóstico Final con Probabilidad de Coincidencias ***")
for rank, disease in enumerate(final_diseases, start=1):
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'breve_descripción'].values[0]
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'tratamiento'].values[0]

    print(f"\n{rank}. **{disease['nombre']}**")
    print(f"   - Probabilidad del modelo: {disease['probabilidad_modelo']:.1f}%")
    print(f"   - Coincidencia con síntomas: {disease['probabilidad_coincidencias']:.1f}% ({disease['síntomas_coincididos']}/{disease['total_síntomas']})")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")


Ingrese sus síntomas: tos, dolor de cabeza

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'dolor de cabeza' coincide con 'dolor_de_cabeza' en el dataset.
Se han identificado 6 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): m
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2.10

Su IMC es 22.68, lo que corresponde a 'peso_normal'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fatiga'? (s/n): s
¿Presenta 'dificultad_respiratoria'? (s/n): s
¿Presenta 'dolor_muscular'? (s/n): n
¿Presenta 'congestión_nasal'? (s/n): s
¿Presenta 'dolor_de_garganta'? (s/n): s
¿Presenta 'escalofríos'? (s/n): s
¿Presenta 'tos_seca'? (s/n): s
¿Presenta 'fiebre_alta'? (s/n): s
¿Presenta 'pérdida_del_olfato_y_gusto'? (s/n): s
¿Presenta 'sibilancias'? (s/n): s
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'mareos'? (s/n): n
¿Presenta 'dolor_en_el_pecho'? (s/n): n
¿Presenta 'estornudos'? (s/n): s
¿Presenta 'secreción_

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Función optimizada para encontrar síntomas válidos
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        s = s.lower().strip()
        if s in symptom_cols:
            sintomas_validos.append(s)
            coincidencias[s] = s
        else:
            match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
            if match:
                sintomas_validos.append(match[0])
                coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
while True:
    intentos = 3
    while intentos > 0:
        entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
        sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)
        sintomas_usuario = [s.strip() for s in sintomas_usuario if s]
        sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.70)

        if sintomas_validos:
            break
        else:
            intentos -= 1
            if intentos > 0:
                print(f"⚠️ No se encontraron coincidencias. Intente nuevamente ({intentos} intentos restantes).")

    print("\nCoincidencias encontradas:")
    for original, match in coincidencias.items():
        print(f"'{original}' coincide con '{match}' en el dataset.")

    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
        threshold = max(0.5, df['match_score'].quantile(0.7))
        df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos con MinMaxScaler
if len(X) > 0:
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest sin validación cruzada
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}
user_answers.update({risk: 0 for risk in risk_cols})
user_answers['hombre' if genero == 'M' else 'mujer'] = 1
user_answers[categoria_imc] = 1

# Seleccionar los síntomas más frecuentes en las enfermedades filtradas
top_symptoms = df_filtrado[symptom_cols].sum().sort_values(ascending=False).index.tolist()

# Asegurar que los síntomas ingresados por el usuario también se pregunten
symptoms_to_ask = list(set(sintomas_validos + top_symptoms[:20]))  # Aumentamos de 20 síntomas

# Establecer un mínimo de preguntas antes de interrumpir
MIN_PREGUNTAS = 10
preguntas_realizadas = 0

print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")

for symptom in symptoms_to_ask:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()

    # Validación de entrada
    while resp not in ["s", "n", "si", "no"]:
        resp = input(f"⚠️ Entrada inválida. ¿Presenta '{symptom}'? (s/n): ").strip().lower()

    user_answers[symptom] = 1 if resp in ["s", "si"] else 0
    preguntas_realizadas += 1

    # Actualizar la predicción con los síntomas ingresados hasta el momento
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]

    # Obtener las 3 enfermedades más probables en este punto
    top3_idx = np.argsort(probs)[-3:][::-1]
    top_final_probabilities = []

    for idx in top3_idx:
        disease_code = clf.classes_[idx]
        disease_name = le.inverse_transform([disease_code])[0]

        # Obtener los síntomas reales de la enfermedad en el dataset (manejo de errores)
        disease_data = df.loc[df['nombre_de_la_enfermedad'] == disease_name, symptom_cols]

        if disease_data.empty or disease_data.iloc[0].sum() == 0:
            total_symptoms = 0
            symptoms_present = []
        else:
            disease_symptoms = disease_data.iloc[0]
            symptoms_present = disease_symptoms[disease_symptoms == 1].index.tolist()
            total_symptoms = len(symptoms_present)

        # Contar coincidencias con los síntomas ingresados por el usuario
        matched_symptoms = sum(user_answers.get(symptom, 0) for symptom in symptoms_present)
        probability_match = (matched_symptoms / total_symptoms) * 100 if total_symptoms > 0 else 0

        # Probabilidad del modelo
        model_probability = probs[idx] * 100

        # Nueva probabilidad combinada
        final_probability = round((model_probability + probability_match) / 2, 1)
        top_final_probabilities.append(final_probability)

    # Si ya se han hecho al menos 10 preguntas y alguna enfermedad tiene una probabilidad combinada ≥ 70%, detener preguntas
    if preguntas_realizadas >= MIN_PREGUNTAS and max(top_final_probabilities) >= 70:
        print("\n🔹 Se ha alcanzado una alta probabilidad de enfermedad. No es necesario más preguntas.\n")
        break

# Obtener las 3 enfermedades más probables después de la fase de preguntas
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Lista de enfermedades finales y sus síntomas asociados
final_diseases = []
for idx in top3_idx:
    disease_code = clf.classes_[idx]
    disease_name = le.inverse_transform([disease_code])[0]

    # Obtener los síntomas reales de la enfermedad en el dataset (manejo de errores)
    disease_data = df.loc[df['nombre_de_la_enfermedad'] == disease_name, symptom_cols]

    if disease_data.empty or disease_data.iloc[0].sum() == 0:
        total_symptoms = 0
        symptoms_present = []
    else:
        disease_symptoms = disease_data.iloc[0]
        symptoms_present = disease_symptoms[disease_symptoms == 1].index.tolist()
        total_symptoms = len(symptoms_present)

    # Contar coincidencias con los síntomas ingresados por el usuario
    matched_symptoms = sum(user_answers.get(symptom, 0) for symptom in symptoms_present)
    probability_match = (matched_symptoms / total_symptoms) * 100 if total_symptoms > 0 else 0

    # Probabilidad del modelo
    model_probability = probs[idx] * 100

    # Nueva probabilidad combinada (redondeada)
    final_probability = round((model_probability + probability_match) / 2, 1)

    final_diseases.append({
        "nombre": disease_name,
        "probabilidad_modelo": round(model_probability, 1),
        "probabilidad_coincidencias": round(probability_match, 1),
        "probabilidad_final": final_probability,
        "síntomas_coincididos": matched_symptoms,
        "total_síntomas": total_symptoms
    })

# Mostrar diagnóstico final con ambas probabilidades combinadas
print("\n*** Diagnóstico Final basado en Modelo + Coincidencia de Síntomas ***")
for rank, disease in enumerate(final_diseases, start=1):
    # Obtener descripción y tratamiento (manejo de errores)
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'breve_descripción']
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'tratamiento']

    desc = desc.values[0] if not desc.empty else "No disponible"
    treat = treat.values[0] if not treat.empty else "No disponible"

    print(f"\n{rank}. **{disease['nombre']}**")
    print(f"   - Probabilidad del modelo: {disease['probabilidad_modelo']:.1f}%")
    print(f"   - Probabilidad basada en síntomas: {disease['probabilidad_coincidencias']:.1f}% ({disease['síntomas_coincididos']}/{disease['total_síntomas']})")
    print(f"   - 🔹 **Probabilidad final combinada**: {disease['probabilidad_final']:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")



Ingrese sus síntomas: tos, dolor de cabeza

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'dolor de cabeza' coincide con 'dolor_de_cabeza' en el dataset.
Se han identificado 6 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): M
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'fiebre'? (s/n): s
¿Presenta 'opresión_en_el_pecho'? (s/n): s
¿Presenta 'dolor_muscular'? (s/n): n
¿Presenta 'escalofríos'? (s/n): s
¿Presenta 'mareos'? (s/n): s
¿Presenta 'dolor_de_garganta'? (s/n): s
¿Presenta 'sibilancias'? (s/n): s
¿Presenta 'dolor_en_el_pecho'? (s/n): n
¿Presenta 'dificultad_respiratoria'? (s/n): s
¿Presenta 'erupción_cutánea'? (s/n): n
¿Presenta 'secreción_nasal_acuosa'? (s/n): s
¿Presenta 'fiebre_alta'? (s/n): s
¿Presenta 'dolor_de_cabeza'? (s/n): s
¿Presenta 'estornudos'? (s/n): s
¿Presenta

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Función optimizada para encontrar síntomas válidos
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        s = s.lower().strip()
        if s in symptom_cols:
            sintomas_validos.append(s)
            coincidencias[s] = s
        else:
            match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
            if match:
                sintomas_validos.append(match[0])
                coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario
while True:
    intentos = 3
    while intentos > 0:
        entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
        sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)
        sintomas_usuario = [s.strip() for s in sintomas_usuario if s]
        sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.70)

        if sintomas_validos:
            break
        else:
            intentos -= 1
            if intentos > 0:
                print(f"⚠️ No se encontraron coincidencias. Intente nuevamente ({intentos} intentos restantes).")

    print("\nCoincidencias encontradas:")
    for original, match in coincidencias.items():
        print(f"'{original}' coincide con '{match}' en el dataset.")

    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
        threshold = max(0.5, df['match_score'].quantile(0.7))
        df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos con MinMaxScaler
if len(X) > 0:
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest sin validación cruzada
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}  # Todos los síntomas empiezan en 0
user_answers.update({risk: 0 for risk in risk_cols})  # Todos los factores de riesgo en 0
user_answers['hombre' if genero == 'M' else 'mujer'] = 1  # Marcamos género
user_answers[categoria_imc] = 1  # Marcamos categoría IMC

# 🔹 Marcar los síntomas ingresados por el usuario como 1
for sintoma in sintomas_validos:
    user_answers[sintoma] = 1  # Se asigna 1 a los síntomas que el usuario ya reportó

# Filtrar enfermedades según factores de riesgo del usuario
factores_usuario = ['hombre' if genero == 'M' else 'mujer', categoria_imc]
df_riesgo = df_filtrado.copy()

for factor in factores_usuario:
    df_riesgo = df_riesgo[df_riesgo[factor] == 1]  # Filtramos solo enfermedades que coincidan

# Si no hay suficientes enfermedades después del filtro, usamos todas las filtradas previamente
if df_riesgo.empty:
    df_riesgo = df_filtrado.copy()

# Seleccionar los síntomas más frecuentes en enfermedades que coinciden con los factores de riesgo
top_symptoms = df_riesgo[symptom_cols].sum().sort_values(ascending=False).index.tolist()

# Asegurar que los síntomas ingresados por el usuario también se pregunten
symptoms_to_ask = list(set(top_symptoms[:20]) - set(sintomas_validos))  # Excluye síntomas ya ingresados por el usuario

# Establecer un mínimo de preguntas antes de interrumpir
MIN_PREGUNTAS = 10
preguntas_realizadas = 0

print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")


for symptom in symptoms_to_ask:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()

    # Validación de entrada
    while resp not in ["s", "n", "si", "no"]:
        resp = input(f"⚠️ Entrada inválida. ¿Presenta '{symptom}'? (s/n): ").strip().lower()

    user_answers[symptom] = 1 if resp in ["s", "si"] else 0
    preguntas_realizadas += 1

    # Actualizar la predicción con los síntomas ingresados hasta el momento
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]

    # Obtener las 3 enfermedades más probables en este punto
    top3_idx = np.argsort(probs)[-3:][::-1]
    top_final_probabilities = []

    for idx in top3_idx:
        disease_code = clf.classes_[idx]
        disease_name = le.inverse_transform([disease_code])[0]

        # Obtener los síntomas reales de la enfermedad en el dataset (manejo de errores)
        disease_data = df.loc[df['nombre_de_la_enfermedad'] == disease_name, symptom_cols]

        if disease_data.empty or disease_data.iloc[0].sum() == 0:
            total_symptoms = 0
            symptoms_present = []
        else:
            disease_symptoms = disease_data.iloc[0]
            symptoms_present = disease_symptoms[disease_symptoms == 1].index.tolist()
            total_symptoms = len(symptoms_present)

        # Contar coincidencias con los síntomas ingresados por el usuario
        matched_symptoms = sum(user_answers.get(symptom, 0) for symptom in symptoms_present)
        probability_match = (matched_symptoms / total_symptoms) * 100 if total_symptoms > 0 else 0

        # Probabilidad del modelo
        model_probability = probs[idx] * 100

        # Nueva probabilidad combinada
        final_probability = round((model_probability + probability_match) / 2, 1)
        top_final_probabilities.append(final_probability)

    # Si ya se han hecho al menos 10 preguntas y alguna enfermedad tiene una probabilidad combinada ≥ 70%, detener preguntas
    if preguntas_realizadas >= MIN_PREGUNTAS and max(top_final_probabilities) >= 70:
        print("\n🔹 Se ha alcanzado una alta probabilidad de enfermedad. No es necesario más preguntas.\n")
        break

# Obtener las 3 enfermedades más probables después de la fase de preguntas
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Lista de enfermedades finales y sus síntomas asociados
final_diseases = []
for idx in top3_idx:
    disease_code = clf.classes_[idx]
    disease_name = le.inverse_transform([disease_code])[0]

    # Obtener los síntomas reales de la enfermedad en el dataset (manejo de errores)
    disease_data = df.loc[df['nombre_de_la_enfermedad'] == disease_name, symptom_cols]

    if disease_data.empty or disease_data.iloc[0].sum() == 0:
        total_symptoms = 0
        symptoms_present = []
    else:
        disease_symptoms = disease_data.iloc[0]
        symptoms_present = disease_symptoms[disease_symptoms == 1].index.tolist()
        total_symptoms = len(symptoms_present)

    # Contar coincidencias con los síntomas ingresados por el usuario
    matched_symptoms = sum(user_answers.get(symptom, 0) for symptom in symptoms_present)
    probability_match = (matched_symptoms / total_symptoms) * 100 if total_symptoms > 0 else 0

    # Probabilidad del modelo
    model_probability = probs[idx] * 100

    # Nueva probabilidad combinada (redondeada)
    final_probability = round((model_probability + probability_match) / 2, 1)

    final_diseases.append({
        "nombre": disease_name,
        "probabilidad_modelo": round(model_probability, 1),
        "probabilidad_coincidencias": round(probability_match, 1),
        "probabilidad_final": final_probability,
        "síntomas_coincididos": matched_symptoms,
        "total_síntomas": total_symptoms
    })

# Mostrar diagnóstico final con ambas probabilidades combinadas
print("\n*** Diagnóstico Final basado en Modelo + Coincidencia de Síntomas ***")
for rank, disease in enumerate(final_diseases, start=1):
    # Obtener descripción y tratamiento (manejo de errores)
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'breve_descripción']
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'tratamiento']

    desc = desc.values[0] if not desc.empty else "No disponible"
    treat = treat.values[0] if not treat.empty else "No disponible"

    print(f"\n{rank}. **{disease['nombre']}**")
    print(f"   - Probabilidad del modelo: {disease['probabilidad_modelo']:.1f}%")
    print(f"   - Probabilidad basada en síntomas: {disease['probabilidad_coincidencias']:.1f}% ({disease['síntomas_coincididos']}/{disease['total_síntomas']})")
    print(f"   - 🔹 **Probabilidad final combinada**: {disease['probabilidad_final']:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")



Ingrese sus síntomas: tos, dolor de cabeza

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
'dolor de cabeza' coincide con 'dolor_de_cabeza' en el dataset.
Se han identificado 6 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 21
Ingrese su género (M/F): m
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'erupción_cutánea'? (s/n): n
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'opresión_en_el_pecho'? (s/n): n
¿Presenta 'secreción_nasal_acuosa'? (s/n): s
¿Presenta 'fiebre_alta'? (s/n): s
¿Presenta 'dolor_muscular'? (s/n): s
¿Presenta 'estornudos'? (s/n): s
¿Presenta 'escalofríos'? (s/n): s
¿Presenta 'congestión_nasal'? (s/n): s
¿Presenta 'tos_seca'? (s/n): s
¿Presenta 'pérdida_del_olfato_y_gusto'? (s/n): s
¿Presenta 'mareos'? (s/n): n
¿Presenta 'dolor_de_garganta'? (s/n): s
¿Presenta 'sibilancias'? (s/n): s
¿Presenta 'vis

In [ ]:
import pandas as pd
import numpy as np
import difflib
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el dataset y limpiar columnas
file_path = "Dataset_Enfermedades.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Definir columnas clave
risk_cols = ['hombre', 'mujer', 'obesidad', 'sobrepeso', 'peso_normal', 'desnutricion']
symptom_cols = [col for col in df.columns if col not in ['nombre_de_la_enfermedad', 'breve_descripción', 'tratamiento'] + risk_cols]

# Función optimizada para encontrar síntomas válidos
def encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.85):
    sintomas_validos = []
    coincidencias = {}
    for s in sintomas_usuario:
        s = s.lower().strip()
        if s in symptom_cols:
            sintomas_validos.append(s)
            coincidencias[s] = s
        else:
            match = difflib.get_close_matches(s, symptom_cols, n=1, cutoff=cutoff)
            if match:
                sintomas_validos.append(match[0])
                coincidencias[s] = match[0]
    return sintomas_validos, coincidencias

# Solicitar síntomas al usuario una única vez
intentos = 3
while intentos > 0:
    entrada_usuario = input("Ingrese sus síntomas: ").lower().strip()
    sintomas_usuario = re.split(r'[,;]\s*', entrada_usuario)
    sintomas_usuario = [s.strip() for s in sintomas_usuario if s]

    # Buscar coincidencias en el dataset
    sintomas_validos, coincidencias = encontrar_sintomas_validos(sintomas_usuario, symptom_cols, cutoff=0.70)

    if sintomas_validos:
        break  # Salimos del bucle si hay síntomas válidos
    else:
        intentos -= 1
        if intentos > 0:
            print(f"⚠️ No se encontraron coincidencias. Intente nuevamente ({intentos} intentos restantes).")
        else:
            print("❌ No se ingresaron síntomas válidos. Terminando ejecución.")
            exit()  # Terminar si no hay síntomas válidos

# Mostrar coincidencias encontradas
print("\nCoincidencias encontradas:")
for original, match in coincidencias.items():
    print(f"'{original}' coincide con '{match}' en el dataset.")

    # Filtrar enfermedades solo una vez y no repetir la solicitud de síntomas
    df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)
    umbral_dinamico = max(0.3, min(0.8, len(sintomas_validos) / len(symptom_cols)))
    threshold = max(0.5, df['match_score'].quantile(umbral_dinamico))
    df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])

    if df_filtrado.empty:
      print("⚠️ No se encontraron enfermedades relacionadas. Sugerimos ingresar más síntomas.")
      exit()  # Terminar si no hay enfermedades válidas

      print(f"✅ Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.\n")

    # 🔹 Filtrado inicial basado en síntomas ingresados por el usuario
    if sintomas_validos:
        df['match_score'] = df[sintomas_validos].sum(axis=1) / len(sintomas_validos)

        # Ajuste del umbral dinámico basado en la cantidad de síntomas ingresados
        umbral_dinamico = max(0.3, min(0.8, len(sintomas_validos) / len(symptom_cols)))
        threshold = max(0.5, df['match_score'].quantile(umbral_dinamico))

        df_filtrado = df[df['match_score'] >= threshold].drop(columns=['match_score'])
    else:
        df_filtrado = df.copy()

    if df_filtrado.empty:
      print("⚠️ No se encontraron enfermedades relacionadas. Sugerimos ingresar más síntomas.")
      exit()


    if len(df_filtrado) == 0:
        print("⚠️ No se encontraron enfermedades relacionadas. Intente nuevamente.")
    else:
        break

print(f"Se han identificado {len(df_filtrado)} enfermedades relacionadas con sus síntomas.")

# Codificar la variable objetivo
le = LabelEncoder()
df_filtrado['enfermedad_cod'] = le.fit_transform(df_filtrado['nombre_de_la_enfermedad'])

# Variables de entrada y salida
X = df_filtrado[symptom_cols + risk_cols]
y = df_filtrado['enfermedad_cod']

# Normalizar datos con MinMaxScaler
if len(X) > 0:
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
else:
    print("⚠️ No hay datos suficientes para normalizar. Verifique el dataset.")
    exit()

# Verificar si hay suficientes datos para entrenamiento
if len(df_filtrado) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
else:
    X_train, y_train = X_scaled, y
    X_test, y_test = X_scaled, y

# Entrenar el modelo de Random Forest sin validación cruzada
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# Guardar modelo y escalador
joblib.dump(clf, "modelo_random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# Solicitar datos del usuario
def solicitar_numero(mensaje, tipo=float):
    while True:
        try:
            return tipo(input(mensaje))
        except ValueError:
            print("Entrada inválida. Inténtelo de nuevo.")

edad = solicitar_numero("Ingrese su edad: ", int)
genero = input("Ingrese su género (M/F): ").strip().upper()
peso = solicitar_numero("Ingrese su peso en kg: ", float)
altura = solicitar_numero("Ingrese su altura en metros: ", float)

# Calcular IMC
imc = peso / (altura ** 2)
categoria_imc = ('desnutricion' if imc < 18.5 else 'peso_normal' if imc < 25 else 'sobrepeso' if imc < 30 else 'obesidad')
print(f"\nSu IMC es {imc:.2f}, lo que corresponde a '{categoria_imc}'.\n")

# Inicializar respuestas del usuario
user_answers = {symptom: 0 for symptom in symptom_cols}  # Todos los síntomas empiezan en 0
user_answers.update({risk: 0 for risk in risk_cols})  # Todos los factores de riesgo en 0
user_answers['hombre' if genero == 'M' else 'mujer'] = 1  # Marcamos género
user_answers[categoria_imc] = 1  # Marcamos categoría IMC

# Inicializar probabilidades dinámicas con los valores iniciales del modelo
probabilidades_dinamicas = {}

# Crear el vector de síntomas del usuario y transformarlo
user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
user_vector_scaled = scaler.transform(user_vector_df)
probs = clf.predict_proba(user_vector_scaled)[0]  # Obtener probabilidades iniciales

# Obtener las 3 enfermedades más probables y asignar sus probabilidades iniciales
top3_idx = np.argsort(probs)[-3:][::-1]  # Índices de las 3 enfermedades más probables

for idx in top3_idx:
    disease_code = clf.classes_[idx]
    disease_name = le.inverse_transform([disease_code])[0]  # Convertir código a nombre
    probabilidades_dinamicas[disease_name] = probs[idx] * 100  # Guardar en porcentaje


# 🔹 Marcar los síntomas ingresados por el usuario como 1
for sintoma in sintomas_validos:
    user_answers[sintoma] = 1  # Se asigna 1 a los síntomas que el usuario ya reportó

# Filtrar enfermedades según factores de riesgo del usuario
factores_usuario = ['hombre' if genero == 'M' else 'mujer', categoria_imc]
df_riesgo = df_filtrado.copy()

for factor in factores_usuario:
    df_riesgo = df_riesgo[df_riesgo[factor] == 1]  # Filtramos solo enfermedades que coincidan

# Si no hay suficientes enfermedades después del filtro, usamos todas las filtradas previamente
if df_riesgo.empty:
    df_riesgo = df_filtrado.copy()

# 🔹 Generación de preguntas dinámicas basadas en síntomas distintivos
sintomas_criticos = []
for enfermedad in df_riesgo['nombre_de_la_enfermedad'].unique():
    disease_data = df[df['nombre_de_la_enfermedad'] == enfermedad][symptom_cols]
    sintomas_unicos = disease_data.loc[:, (disease_data == 1).all()].columns.tolist()
    sintomas_criticos.extend(sintomas_unicos)

# Priorizar síntomas distintivos antes que síntomas frecuentes
symptoms_to_ask = list(set(sintomas_criticos + top_symptoms[:20]) - set(sintomas_validos))

if not symptoms_to_ask:
    print("\n🔹 No hay más síntomas relevantes. Se agregan preguntas de respaldo.\n")
    symptoms_to_ask = list(set(symptom_cols) - set(sintomas_validos))[:10]


# Establecer un mínimo de preguntas antes de interrumpir
MIN_PREGUNTAS = 10
preguntas_realizadas = 0

print("\nPor favor responda las siguientes preguntas con 's' (sí) o 'n' (no):\n")

for symptom in symptoms_to_ask:
    resp = input(f"¿Presenta '{symptom}'? (s/n): ").strip().lower()

    # Validación de entrada
    while resp not in ["s", "n", "si", "no"]:
        resp = input(f"⚠️ Entrada inválida. ¿Presenta '{symptom}'? (s/n): ").strip().lower()

    user_answers[symptom] = 1 if resp in ["s", "si"] else 0
    preguntas_realizadas += 1

# 🔹 Corrección: Manejo de la selección de preguntas dinámicas

# Generar lista de preguntas prioritarias basadas en síntomas distintivos y frecuentes
sintomas_criticos = []
for enfermedad in df_riesgo['nombre_de_la_enfermedad'].unique():
    disease_data = df[df['nombre_de_la_enfermedad'] == enfermedad][symptom_cols]
    sintomas_unicos = disease_data.loc[:, (disease_data == 1).all()].columns.tolist()
    sintomas_criticos.extend(sintomas_unicos)

# Priorizar síntomas distintivos antes que síntomas frecuentes
symptoms_to_ask = list(set(sintomas_criticos + top_symptoms[:20]) - set(sintomas_validos))

# 🔹 Corrección: Manejo de preguntas de respaldo si no hay suficientes síntomas distintivos
if not symptoms_to_ask:
    print("\n🔹 No hay más síntomas relevantes. Se agregan preguntas de respaldo.\n")
    symptoms_to_ask = list(set(symptom_cols) - set(sintomas_validos))[:10]

# 🔹 Corrección: Ajuste dinámico en la eliminación de enfermedades
for enfermedad in list(probabilidades_dinamicas.keys()):
    if user_answers[symptom] == 0:  # Si el usuario dice "No"
        factor_reduccion = max(0.85, 1 - (preguntas_realizadas / 100))  # 🔹 Se reduce de forma más controlada
        probabilidades_dinamicas[enfermedad] *= factor_reduccion
    elif user_answers[symptom] == 1:  # Si el usuario dice "Sí"
        probabilidades_dinamicas[enfermedad] *= 1.05

    # 🔹 Corrección: Evitar eliminaciones prematuras antes de 8 preguntas
    if preguntas_realizadas < 8:
        continue

    # 🔹 Eliminación de enfermedades con probabilidad menor al 20% en fases avanzadas
    if probabilidades_dinamicas[enfermedad] < 20:
        del probabilidades_dinamicas[enfermedad]

    # Actualizar la predicción con los síntomas ingresados hasta el momento
    user_vector_df = pd.DataFrame([user_answers], columns=X.columns)
    user_vector_scaled = scaler.transform(user_vector_df)
    probs = clf.predict_proba(user_vector_scaled)[0]


    # Obtener las 3 enfermedades más probables en este punto
    top3_idx = np.argsort(probs)[-3:][::-1]
    top_final_probabilities = []

    for idx in top3_idx:
        disease_code = clf.classes_[idx]
        disease_name = le.inverse_transform([disease_code])[0]

        # Obtener los síntomas reales de la enfermedad en el dataset (manejo de errores)
        disease_data = df.loc[df['nombre_de_la_enfermedad'] == disease_name, symptom_cols]

        if disease_data.empty or disease_data.iloc[0].sum() == 0:
            total_symptoms = 0
            symptoms_present = []
        else:
            disease_symptoms = disease_data.iloc[0]
            symptoms_present = disease_symptoms[disease_symptoms == 1].index.tolist()
            total_symptoms = len(symptoms_present)

        # Contar coincidencias con los síntomas ingresados por el usuario
        matched_symptoms = sum(user_answers.get(symptom, 0) for symptom in symptoms_present)
        probability_match = (matched_symptoms / total_symptoms) * 100 if total_symptoms > 0 else 0

        # Probabilidad del modelo
        model_probability = probs[idx] * 100

        # Nueva probabilidad combinada
        final_probability = round((model_probability + probability_match) / 2, 1)
        top_final_probabilities.append(final_probability)

    # Si ya se han hecho al menos 10 preguntas y alguna enfermedad tiene una probabilidad combinada ≥ 70%, detener preguntas
    if preguntas_realizadas >= MIN_PREGUNTAS and max(top_final_probabilities) >= 70:
        print("\n🔹 Se ha alcanzado una alta probabilidad de enfermedad. No es necesario más preguntas.\n")
        break

# Obtener las 3 enfermedades más probables después de la fase de preguntas
probs = clf.predict_proba(user_vector_scaled)[0]
top3_idx = np.argsort(probs)[-3:][::-1]

# Lista de enfermedades finales y sus síntomas asociados
final_diseases = []
for idx in top3_idx:
    disease_code = clf.classes_[idx]
    disease_name = le.inverse_transform([disease_code])[0]

    # Obtener los síntomas reales de la enfermedad en el dataset (manejo de errores)
    disease_data = df.loc[df['nombre_de_la_enfermedad'] == disease_name, symptom_cols]

    if disease_data.empty or disease_data.iloc[0].sum() == 0:
        total_symptoms = 0
        symptoms_present = []
    else:
        disease_symptoms = disease_data.iloc[0]
        symptoms_present = disease_symptoms[disease_symptoms == 1].index.tolist()
        total_symptoms = len(symptoms_present)

    # Contar coincidencias con los síntomas ingresados por el usuario
    matched_symptoms = sum(user_answers.get(symptom, 0) for symptom in symptoms_present)
    probability_match = (matched_symptoms / total_symptoms) * 100 if total_symptoms > 0 else 0

    # Probabilidad del modelo
    model_probability = probs[idx] * 100

    # Nueva probabilidad combinada (redondeada)
    final_probability = round((model_probability + probability_match) / 2, 1)

    final_diseases.append({
        "nombre": disease_name,
        "probabilidad_modelo": round(model_probability, 1),
        "probabilidad_coincidencias": round(probability_match, 1),
        "probabilidad_final": final_probability,
        "síntomas_coincididos": matched_symptoms,
        "total_síntomas": total_symptoms
    })

# Mostrar diagnóstico final con ambas probabilidades combinadas
print("\n*** Diagnóstico Final basado en Modelo + Coincidencia de Síntomas ***")
for rank, disease in enumerate(final_diseases, start=1):
    # Obtener descripción y tratamiento (manejo de errores)
    desc = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'breve_descripción']
    treat = df.loc[df['nombre_de_la_enfermedad'] == disease["nombre"], 'tratamiento']

    desc = desc.values[0] if not desc.empty else "No disponible"
    treat = treat.values[0] if not treat.empty else "No disponible"

    print(f"\n{rank}. **{disease['nombre']}**")
    print(f"   - Probabilidad del modelo: {disease['probabilidad_modelo']:.1f}%")
    print(f"   - Probabilidad basada en síntomas: {disease['probabilidad_coincidencias']:.1f}% ({disease['síntomas_coincididos']}/{disease['total_síntomas']})")
    print(f"   - 🔹 **Probabilidad final combinada**: {disease['probabilidad_final']:.1f}%")
    print(f"   *Descripción:* {desc}")
    print(f"   *Tratamiento:* {treat}\n")



Ingrese sus síntomas: tos, dolor de cabeza

Coincidencias encontradas:
'tos' coincide con 'tos' en el dataset.
Se han identificado 6 enfermedades relacionadas con sus síntomas.
Ingrese su edad: 2
Ingrese su género (M/F): M
Ingrese su peso en kg: 100
Ingrese su altura en metros: 2

Su IMC es 25.00, lo que corresponde a 'sobrepeso'.


Por favor responda las siguientes preguntas con 's' (sí) o 'n' (no):

¿Presenta 'erupción_cutánea'? (s/n): n
¿Presenta 'fiebre'? (s/n): s
¿Presenta 'opresión_en_el_pecho'? (s/n): n
¿Presenta 'secreción_nasal_acuosa'? (s/n): s
¿Presenta 'fiebre_alta'? (s/n): s
¿Presenta 'parches_rojos_con_escamas_plateadas'? (s/n): n
¿Presenta 'dolor_muscular'? (s/n): s
¿Presenta 'palidez'? (s/n): n
¿Presenta 'picazón'? (s/n): n
¿Presenta 'estornudos'? (s/n): s
¿Presenta 'conjuntivitis'? (s/n): n
¿Presenta 'escalofríos'? (s/n): s
¿Presenta 'congestión_nasal'? (s/n): s
¿Presenta 'palpitaciones'? (s/n): n
¿Presenta 'tos_seca'? (s/n): s
¿Presenta 'pérdida_del_olfato_y_gusto'? (